In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import html2text
import httpx
import json
import lxml
import re
import glob

from lxml import html
from extractcontent3 import ExtractContent
from readability.readability import Document
from trafilatura import fetch_url, extract

from tqdm import tqdm

In [3]:
def extractcontent(html):
    extractor = ExtractContent()

    opt = {"threshold":50}
    extractor.set_option(opt)

    extractor.analyse(html)
    text, title = extractor.as_text().strip()
    html, title = extractor.as_html()
    title = extractor.extract_title(html)
    return title, text

def readability_html2text(html):
    doc = Document(html)
    title = doc.title()
    summary = doc.summary()
    summary_text = html2text.html2text(summary).strip()
    return title, summary_text

def readability_lxml(html):
    doc = Document(html)
    title = doc.title()
    summary = doc.summary()
    summary_text = lxml.html.fromstring(summary).text_content().strip()
    return title, summary_text

def tura(html):
    doc = Document(html)
    title = doc.title()
    text = extract(html)
    return title, text

def extract_content_text(html):
    return readability_lxml(html)
    #return tura(html)

In [4]:
!ls ../../data/fc2b/html

1    120  142  164  186  207  229  250	272  294  315  337  36	 381  59  80
10   121  143  165  187  208  23   251	273  295  316  338  360  382  6   81
100  122  144  166  188  209  230  252	274  296  317  339  361  39   60  82
101  123  145  167  189  21   231  253	275  297  318  34   362  4    61  83
102  124  146  168  19	 210  232  254	276  298  319  340  363  40   62  84
103  125  147  169  190  211  233  255	277  299  32   341  364  41   63  85
104  126  148  17   191  212  234  256	278  3	  320  342  365  42   64  86
105  127  149  170  192  213  235  257	279  30   321  343  366  43   65  87
106  128  15   171  193  214  236  258	28   300  322  344  367  44   66  88
107  129  150  172  194  215  237  259	280  301  323  345  368  45   67  89
108  13   151  173  195  216  238  26	281  302  324  346  369  46   68  9
109  130  152  174  196  217  239  260	282  303  325  347  37	 47   69  90
11   131  153  175  197  218  24   261	283  304  326  348  370  48   7   91
110  132  154  176

In [5]:
headers = {}
cookies = {'age_check': '1'}

In [6]:
toxicity_genres = (333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 378, 379, 381)

In [7]:
genre_list = httpx.get('https://blog.fc2.com/genrelist/', headers=headers, cookies=cookies)
genre_groups = html.fromstring(genre_list.text).xpath('//*[@class="genre_group"]')
genre_data = []
for group in genre_groups:
    genre = group.xpath('./h3/a')[0]
    genre_id = genre.attrib['href'].strip().split('/')[-2]
    genre_text = genre.text.strip()
    
    children = []
    subgenres = group.xpath('./ul/li/a')
    for node in subgenres:
        subgenre_id = re.findall(r'/subgenre/([0-9]+)/', node.attrib['href'])[0]
        subgenre_text = ''.join(node.itertext()).strip()
        children.append({
            'id': subgenre_id,
            'name': subgenre_text
        })

    genre_data.append({
        'id': genre_id,
        'name': genre_text,
        'subgenre': children
    })
    
subgenre2genre = {subgenre['id']: (genre['id'], genre['name']) for genre in genre_data for subgenre in genre['subgenre']}
subgenre2name = {subgenre['id']: subgenre['name'] for genre in genre_data for subgenre in genre['subgenre']}

In [8]:
data = []
for path in tqdm(glob.glob('../../data/fc2b/**/*.html', recursive=True)[:10]):
    entry = path.split('/')[-1]
    subgenre_id = path.split('/')[-2]
    toxicity = 1 if genre_id in toxicity_genres else 0
    try:
        title, summary = extract_content_text(open(path).read())
        if summary.strip():
            data.append({
                'blog': entry.split('#')[0],
                'entry': entry.split('#')[1],
                'genre_id': subgenre2genre[subgenre_id][0],
                'genre_name': subgenre2genre[subgenre_id][1],
                'subgenre_id': subgenre_id,
                'subgenre_name': subgenre2name[subgenre_id],
                'toxicity': toxicity,
                'title': title,
                'text': summary
            })
    except Exception as exc:
        print(f"{type(exc).__name__}. {str(exc)} {path}")

 50%|██████████████████████████████████████████████████████████████████████████████████████                                                                                      | 5/10 [00:00<00:00, 35.20it/s]

ParserError. Document is empty ../../data/fc2b/html/135/haircutfan.blog.fc2.com#blog-entry-48156.html
ParserError. Document is empty ../../data/fc2b/html/135/wwide1225.blog.fc2.com#blog-entry-5030.html
ParserError. Document is empty ../../data/fc2b/html/135/life39naturel.blog.fc2.com#blog-entry-5203.html


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.47it/s]


In [9]:
data

[{'blog': 'dointhemod.blog100.fc2.com',
  'entry': 'blog-entry-1537.html',
  'genre_id': '15',
  'genre_name': 'ファッション・ブランド',
  'subgenre_id': '135',
  'subgenre_name': '海外ブランド',
  'toxicity': 0,
  'title': "Art Gallery Clothing Knitted Polo Shirt 〜 アートギャラリー クロージング “WHARTON” ニットポロシャツ - ドゥインザモッド ブログ | オンラインモッズショップ DoiN' THe MoD の最新情報・名古屋MODシーン情報",
  'text': '英国アートギャラリー クロージングから“WHARTON”ニットポロシャツのネイビーが新入荷。モッズにとって鹿の子ポロシャツといえば2本のティップライン入りが定番。こちらの“WHARTON”ニットポロシャツには襟と前立てに2本のハンド風ステッチが施される。スポーティでカジュアルな鹿の子ポロに対して上品でシックな印象のニットポロにステッチ仕上げはとても相性が良い。そしてModが好むニットポロの基本ディテールであるヴィンテージポロのような存在感のある襟、クラシックな3つボタンを装備。飾りすぎない控えめなデザインだからこそ合わせやすいという利点も。オンラインショップhttps://dointhemod.com/?pid=183773231'},
 {'blog': 'shirokiyagofukuten.blog.fc2.com',
  'entry': 'blog-entry-4083.html',
  'genre_id': '15',
  'genre_name': 'ファッション・ブランド',
  'subgenre_id': '135',
  'subgenre_name': '海外ブランド',
  'toxicity': 0,
  'title': '白木屋呉服店 赤嶺猛さんの琉球絣の帯合わせ',
  'text': '第六千三百八十六回目は、赤嶺猛さんの琉球絣の帯合わせです。今日は使い残し画像です。いちばん上の写真は、菱一の押絵の名古屋帯を合わせてみまし

In [10]:
with open('/tmp/fc2b.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)